# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
import pickle
import warnings
from sqlalchemy import create_engine

warnings.filterwarnings("ignore")
# import NLP libraries
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database
db_engine = create_engine('sqlite:///DisasterResponse.db')
df_table_data = pd.read_sql_table('DisasterResponse', db_engine)
X = df_table_data['message']
Y = df_table_data.drop(['id', 'message', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize_util(input_text):
    # Define url pattern
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    matched_url_patterns = re.findall(url_regex, input_text)
    for url_val in matched_url_patterns:
        input_text = input_text.replace(url_val, "urlplaceholder")
    
    # tokenize sentences
    tokens = word_tokenize(input_text)
    lemmatizer_instance = WordNetLemmatizer()
    
    tokenized_data = [lemmatizer_instance.lemmatize(tok).lower().strip() for tok in tokens]
    
    STOPWORDS = list(set(stopwords.words('english')))
    tokenized_data = [token for token in tokenized_data if token not in STOPWORDS]
    
    return tokenized_data

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
def pipeline_builder():
    
    # build NLP pipeline
    pipeline = Pipeline([
        ('vec', CountVectorizer(tokenizer=tokenize_util)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators = 100, n_jobs = 6)))
    ])
    
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline = pipeline_builder()
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vec', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
def report_builder(pipeline, X_test, y_test):
    
    y_prediction = pipeline.predict(X_test)
    
    # build classification report on every column
    performances_arr = []
    for iter in range(len(y_test.columns)):
        performances_arr.append([f1_score(y_test.iloc[:, iter].values, y_prediction[:, iter], average='micro'),
                             precision_score(y_test.iloc[:, iter].values, y_prediction[:, iter], average='micro'),
                             recall_score(y_test.iloc[:, iter].values, y_prediction[:, iter], average='micro')])
    
    performances_arr = pd.DataFrame(performances_arr, columns=['f1 score', 'precision', 'recall'],
                                index = y_test.columns)   
    return performances_arr
report_builder(pipeline, X_test, y_test)

,f1 score,precision,recall
related,0.810650,0.810650,0.810650
request,0.898535,0.898535,0.898535
offer,0.994507,0.994507,0.994507
aid_related,0.785475,0.785475,0.785475
medical_help,0.923253,0.923253,0.923253
medical_products,0.952701,0.952701,0.952701
search_and_rescue,0.973299,0.973299,0.973299
security,0.979249,0.979249,0.979249
military,0.966738,0.966738,0.966738
child_alone,1.000000,1.000000,1.000000


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
finetuned_parameters = {'clf__estimator__max_features':['sqrt', 0.5],
              'clf__estimator__n_estimators':[50, 100]}

cross_validator = GridSearchCV(estimator=pipeline, param_grid = finetuned_parameters, cv = 5, n_jobs = 6)
cross_validator.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.